<font size="6"> **Portfolio Optimization** </font>

In [1]:
%%capture
%run 00_portopt_data.ipynb

In [2]:
import datetime as dt

from tqdm import tqdm

from src.mle_quant_utils import portopt_utils
from src.portfolio_opt import cvx_opt_port

In [3]:
N_DAYS_DELAY = int(cfg['backtesting']['n_days_delay'])
OPT_CONST = cfg['backtesting']['opt_port']
LAMBDA_REG = float(OPT_CONST['lambda_reg'])

In [4]:
print('n_days_delay:', N_DAYS_DELAY)
print('opt constraints params:', OPT_CONST)

n_days_delay: 2
opt constraints params: {'risk_aversion': '1e-6', 'min_mkt_cap': '1e8', 'lambda_reg': '0.5', 'risk_cap': '0.05', 'factor_max': '10.0', 'factor_min': '-10.0', 'weights_max': '0.55', 'weights_min': '-0.55'}


# Statistical Risk Model

In [5]:
risk_model.keys()

dict_keys(['idiosyncratic_var_vector', 'factor_var_vector', 'factor_betas', 'factor_returns'])

In [6]:
risk_model['factor_betas'].shape

(490, 20)

In [7]:
risk_model['factor_cov_matrix'] = np.diag(risk_model['factor_var_vector']['0'].values)
risk_model['factor_cov_matrix'].shape

(20, 20)

In [8]:
risk_model['idiosyncratic_var_vector'].shape

(490, 1)

# Portfolio Optimization

## No Transaction Costs

In [9]:
opt_engine = cvx_opt_port.OptimalHoldingsRegualization(lambda_reg=LAMBDA_REG)

In [10]:
bkt_dates = test_dates.unique()
bkt_daily_returns = daily_returns.loc[bkt_dates].copy()
bkt_daily_adv = adv.loc[bkt_dates].copy()

In [11]:
res_simple_avg_combi = portopt_utils.run_backtesting(opt_engine, False, B_alpha['avg_combi_alpha'], risk_model, bkt_daily_returns, bkt_daily_adv,
                                                     n_days_delay=N_DAYS_DELAY)
pnl_simple_avg_combi, w_simple_avg_combi = res_simple_avg_combi

Opt portfolio: 100%|██████████| 149/149 [00:45<00:00,  3.24portfolio/s]


In [12]:
res_simple_ml = portopt_utils.run_backtesting(opt_engine, False, B_alpha['ml_alpha'], risk_model, bkt_daily_returns, bkt_daily_adv,
                                              n_days_delay=N_DAYS_DELAY)
pnl_simple_ml, w_simple_ml = res_simple_ml

Opt portfolio: 100%|██████████| 149/149 [00:45<00:00,  3.26portfolio/s]


In [13]:
res_simple_1yrmon = portopt_utils.run_backtesting(opt_engine, False, B_alpha['Momentum_1YR_Smoothed'], risk_model, bkt_daily_returns, bkt_daily_adv, n_days_delay=N_DAYS_DELAY)
pnl_simple_1yrmon, w_simple_1yrmon = res_simple_1yrmon

Opt portfolio: 100%|██████████| 149/149 [00:47<00:00,  3.16portfolio/s]


In [14]:
res_simple_5dmrev = portopt_utils.run_backtesting(opt_engine, False, B_alpha['Mean_Reversion_Sector_Neutral_Smoothed'], risk_model, bkt_daily_returns, bkt_daily_adv, n_days_delay=N_DAYS_DELAY)
pnl_simple_5dmrev, w_simple_5dmrev = res_simple_5dmrev

Opt portfolio: 100%|██████████| 149/149 [01:00<00:00,  2.46portfolio/s]


In [15]:
pnl_simple_avg_combi['port_name'] = 'simple_avg_combi'
pnl_simple_5dmrev['port_name'] = 'Mean_Reversion_Sector_Neutral_Smoothed'
pnl_simple_1yrmon['port_name'] = 'Momentum_1YR_Smoothed'
pnl_simple_ml['port_name'] = 'simple_ml'
pnl_simple_app = pd.concat([ pnl_simple_1yrmon, pnl_simple_5dmrev, pnl_simple_avg_combi, pnl_simple_ml ], axis=0)

In [16]:
pnl_simple_app.groupby('port_name')[['accum_total', 'accum_transaction_cost']].last()

,accum_total,accum_transaction_cost
port_name,,
Mean_Reversion_Sector_Neutral_Smoothed,-0.006041,0.0
Momentum_1YR_Smoothed,0.072770,0.0
simple_avg_combi,0.041541,0.0
simple_ml,0.043656,0.0


In [17]:
pnl_simple_app.groupby('port_name')['daily_pnl'].describe()

,count,mean,std,min,25%,50%,75%,max
port_name,,,,,,,,
Mean_Reversion_Sector_Neutral_Smoothed,149.0,-0.000041,0.003892,-0.012430,-0.002523,-0.000054,0.001998,0.014663
Momentum_1YR_Smoothed,149.0,0.000488,0.005876,-0.017641,-0.002813,0.001022,0.004125,0.014700
simple_avg_combi,149.0,0.000279,0.003982,-0.011773,-0.002395,0.000589,0.003313,0.009001
simple_ml,149.0,0.000293,0.003797,-0.009261,-0.002145,0.000057,0.003133,0.010148


## Transaction Costs

In [18]:
from src.mle_quant_utils import portopt_utils
from src.portfolio_opt import cvx_opt_port

opt_engine = cvx_opt_port.OptimalHoldingsRegualization(lambda_reg=LAMBDA_REG)

In [19]:
res_tc_avg_combi = portopt_utils.run_backtesting(opt_engine, True, B_alpha['avg_combi_alpha'], risk_model, bkt_daily_returns, bkt_daily_adv,
                                                 n_days_delay=N_DAYS_DELAY)
pnl_tc_avg_combi, w_tc_avg_combi = res_tc_avg_combi

Opt portfolio: 100%|██████████| 149/149 [00:52<00:00,  2.83portfolio/s]


In [20]:
res_tc_ml = portopt_utils.run_backtesting(opt_engine, True, B_alpha['ml_alpha'], risk_model, bkt_daily_returns, bkt_daily_adv, n_days_delay=N_DAYS_DELAY)
pnl_tc_ml, w_tc_ml = res_tc_ml

Opt portfolio: 100%|██████████| 149/149 [00:50<00:00,  2.93portfolio/s]


In [21]:
pnl_tc_avg_combi['port_name'] = 'tc_avg_combi'
pnl_tc_ml['port_name'] = 'tc_ml'

In [22]:
pnl_tc_app = pd.concat([ pnl_tc_avg_combi, pnl_tc_ml], axis=0)
pnl_simple_app = pd.concat([pnl_simple_avg_combi, pnl_simple_ml], axis=0)

In [23]:
pnl_tc_app.groupby('port_name')[['accum_total', 'accum_transaction_cost']].last()

,accum_total,accum_transaction_cost
port_name,,
tc_avg_combi,0.04147,0.000071
tc_ml,0.04356,0.000097


In [24]:
pnl_tc_app.groupby('port_name')['daily_pnl'].describe()

,count,mean,std,min,25%,50%,75%,max
port_name,,,,,,,,
tc_avg_combi,149.0,0.000279,0.003982,-0.011773,-0.002395,0.000589,0.003313,0.009001
tc_ml,149.0,0.000293,0.003797,-0.009261,-0.002145,0.000057,0.003133,0.010148


# Write Results

## PnL Results

In [25]:
pnl_app = pd.concat([pnl_simple_ml, pnl_simple_avg_combi, pnl_tc_ml, pnl_tc_avg_combi], axis=0)
pd.concat([pnl_app.head(2), pnl_app.tail(2)])

,returns_date,daily_pnl,daily_transaction_cost,daily_total,accum_total,accum_transaction_cost,port_name
date,,,,,,,
2015-05-26 00:00:00+00:00,2015-05-28 00:00:00+00:00,0.001652,0.000000e+00,0.001652,0.001652,0.000000,simple_ml
2015-05-27 00:00:00+00:00,2015-05-29 00:00:00+00:00,0.001363,0.000000e+00,0.001363,0.003015,0.000000,simple_ml
2015-12-22 00:00:00+00:00,2015-12-24 00:00:00+00:00,-0.001911,4.571629e-07,-0.001911,0.038125,0.000070,tc_avg_combi
2015-12-23 00:00:00+00:00,2015-12-28 00:00:00+00:00,0.003345,3.887931e-07,0.003345,0.041470,0.000071,tc_avg_combi


In [26]:
pnl_app.to_csv(OUTPATH1 + OUTFILE1)

## Daily Portfolio Weights Dictionary

In [27]:
models_dict = {'simple_ml_avg_combi': w_simple_avg_combi,
               'simple_ml': w_simple_ml,
               'tc_avg_combi': w_tc_avg_combi,
               'tc_ml': w_tc_ml}

with open(OUTPATH2 + OUTFILE2, 'wb') as file:
    pickle.dump(models_dict, file)

## Portfolio Optimization Metadata

In [28]:
opt_metadata = {"alpha_model": RISK_MODEL_VERSION,
               "risk_model": MLALPHA_MODEL_VERSION,
                "opt_constraints": OPT_CONST,
                "backtesting_start_dt": test_start,
                "backtesting_end_dt": test_end,
                "n_days_delay": N_DAYS_DELAY
               }

with open(OUTPATH2 + f'{OUTFILE3}', 'wb') as file:
    pickle.dump(opt_metadata, file)